In [9]:
### 그랩컷 = 전경과 객체를 분리해준다.
import cv2
import numpy as np

# 이미지 불러오기
img = cv2.imread('./dataset/before/새.png')
img = cv2.resize(img, (500, 500))

img_draw = img.copy()
img_draw = cv2.resize(img_draw, (500, 500))
# 마스크 생성
mask = np.zeros(img.shape[:2], dtype=np.uint8)
# 사각형 영역 좌표 초기화
rect = [0, 0, 0, 0]
# 그랩컷의 초기 모드 = 재시도
mode = cv2.GC_EVAL

# 함수 내에서 사용할 배경과 전경 모델 임시 배열 버퍼
bgdmodel = np.zeros((1, 65), np.float64)
fgdmodel = np.zeros((1, 65), np.float64)

# 마우스 이벤트 처리 함수
def onMouse(event, x, y, flags, param):
    global mouse_mode, rect, mask, mode
    
    # 마우스 왼쪽 클릭하면
    if event == cv2.EVENT_LBUTTONDOWN:
        # 아무 키도 눌리지 않은 경우
        if flags <= 1:
            # 드래그 시작 ( 사각형 그리기 모드 )
            mode = cv2.GC_INIT_WITH_RECT
            # 시작 좌표를 x, y로 저장
            rect[:2] = x, y

    # 만약 마우스 왼쪽 누른 상태에서 움직이면        
    elif event == cv2.EVENT_MOUSEMOVE and flags & cv2.EVENT_FLAG_LBUTTON:
        # 사각형 그리기 모드로 드래그 진행중이면
        if mode == cv2.GC_INIT_WITH_RECT:
            # 그리고 있는 이미지 복사해서 그 위에 사각형 그리는걸 보여주기
            img_temp = img.copy()
            cv2.rectangle(img_temp, (rect[0], rect[1]), (x, y), (0,255,0), 2)
            cv2.imshow('img', img_temp)

        # 만약 키가 눌려있다면 마스크 모드 시작!
        elif flags > 1:
            mode = cv2.GC_INIT_WITH_MASK
            
            # ctrl 키가 눌린 경우
            if flags & cv2.EVENT_FLAG_CTRLKEY:
                # 흰색 점 그리기
                cv2.circle(img_draw, (x, y), 3, (255, 255, 255), -1)
                # 클릭한 좌표를 확실한 전경으로 선택
                cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)
            # 쉬프트키가 눌린경우
            if flags & cv2.EVENT_FLAG_SHIFTKEY:
                # 검정색 점 그리기
                cv2.circle(img_draw, (x, y), 3, (0,0,0), -1)
                # 클릭한 좌표를 확실한 배경으로 선택
                cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1)
            cv2.imshow('img', img_draw)

    # 마우스 오른쪽 버튼에서 떼면
    elif event == cv2.EVENT_LBUTTONUP:
        # 사각형 그리는 모드중이면
        if mode == cv2.GC_INIT_WITH_RECT:
            # x, y 좌표 수집
            rect[2:] = x, y
            # 사각형을 화면에 그려서 출력하기
            cv2.rectangle(img_draw, (rect[0], rect[1]), (x, y), (255,0,0), 2)
            cv2.imshow('img', img_draw)
        
        # 그랩컷 적용 : grabCut(입력 영상, 입력영상과 크기가 같은 배열로 배경과 전경을 구분하는 값을 가지고 있음, 전경의 사각형 좌표, 임시 배열 버퍼, 반복횟수, 동장 방법)
        cv2.grabCut(img, mask, tuple(rect), bgdmodel, fgdmodel, 1, mode)
        img2 = img.copy()
        # 마스크에 확실한 배경(GC_BGD), 또는 아마도 배경(GC_PB_BGD)를 0으로 채우기
        img2[(mask==cv2.GC_BGD) | (mask==cv2.GC_PR_BGD)] = 0
        # 최종 결과 출력
        cv2.imshow('grabcut', img2)
        # 모드 리셋
        mode = cv2.GC_EVAL

# 초기화면 출력
cv2.imshow('img', img)
# 마우스 이벤트 등록
cv2.setMouseCallback('img', onMouse)
# esc 누를때까지 계속 작동하기
while True:
    if cv2.waitKey(0) & 0xFF == 27:
        break
cv2.destroyAllWindows()